In [8]:
import matplotlib

# Memory leak with Matplotlib when in interactive mode and writing 1000+ plots
matplotlib.use('agg')
matplotlib.interactive(False)

import obspy, os, glob

CSV = 'Benz_catalog.csv' # 'OK_2014-2015-2016.csv' # 
FOLDER_NAME = 'modified_benz_without_08_2014'

# In seconds
DURATION = 20 

# Seconds before event
PRE_PADDING = 10

# Seconds after event
POST_PADDING = 10

DATA_PATH = os.path.join(os.getcwd(), 'data')
SPECTROGRAM_PATH = os.path.join(os.getcwd(), f'spectrograms/{FOLDER_NAME}')

stream_paths = glob.glob(os.path.join(DATA_PATH, 'train_mseed/*.mseed'))
stream_paths += glob.glob(os.path.join(DATA_PATH, 'test_mseed/*.mseed'))

In [9]:
testing_paths = ['/data/notebooks/dataset_oklahoma/data/train_mseed/GSOK029_8-2014.mseed']

for path in testing_paths:
    stream_paths.remove(path)

In [10]:
# IMPORT SEISMIC TOOLBOX CODE
import sys
sys.path.insert(0, '/data/seismic_toolbox')

In [11]:
import seismic_code.spectrograms as spectro
import seismic_code.filter as filt
from seismic_code import helpers
from seismic_code.spectrograms import write_spectrogram

In [12]:
from pathlib import Path
from datetime import date
from collections import defaultdict
from obspy import read

class StreamPath:
    def __init__(self, path):
        self.raw_path = path
        self.path = Path(path)
        self.station = self.path.parts[-1].split('_')[0]
        date_str = self.path.parts[-1].split('_')[1].replace('.mseed', "")
        month, year = date_str.split('-')
        self.date_start = date(int(year), int(month), day=1)
        
    def load(self):
        """ Loads a stream from the path """
        return read(self.raw_path)
        
    def __str__(self):
        return self.raw_path
    
paths = list(map(StreamPath, stream_paths))
#s1_date_paths = {stream.date_start: stream for stream in paths if stream.station == 'GSOK027'}
s1_date_paths = {stream.date_start: stream for stream in paths if stream.station == 'GSOK029'}

In [13]:
from obspy import Stream
from operator import add
from functools import reduce
from obspy import read

# Combine all streams into one
# stream = reduce(add, map(read, stream_paths))

In [14]:
import pandas as pd

quake_csv = pd.read_csv(os.path.join(DATA_PATH, CSV))

In [15]:
def change_column_name(df, column_index, new_name):
    columns = df.columns.values
    columns[column_index] = new_name
    df.columns = columns
    
change_column_name(quake_csv, 0, 'EventNum')

quake_csv.head()

,EventNum,Date,Time,Magnitude,Variance,origintime,utc_timestamp
0,0,02/15/2014,00:02:41,0.37,0.95,2014-02-15T00:02:41.000000Z,1.392423e+09
1,1,02/15/2014,00:03:45,-0.44,0.86,2014-02-15T00:03:45.000000Z,1.392423e+09
2,2,02/15/2014,00:08:07,-0.18,0.93,2014-02-15T00:08:07.000000Z,1.392423e+09
3,3,02/15/2014,00:12:52,0.10,0.93,2014-02-15T00:12:52.000000Z,1.392423e+09
4,4,02/15/2014,00:14:09,-0.47,0.89,2014-02-15T00:14:09.000000Z,1.392423e+09


In [16]:
from itertools import islice
from obspy import UTCDateTime

def gen_row_date(df):
    for index, row in df.iterrows():
        year, month, day = row.origintime[:10].split('-')
        yield date(int(year), int(month), day=1), row
        
        
def gen_filter_waves(df, date_paths, pre_padding=PRE_PADDING, post_padding=POST_PADDING, shift_left=0, shift_right=0, offset=1):
    """
    :shift_left: make copies of event by shifting seconds in the left direction
    :shift_right: make copies of event by shifting seconds in the right direction
    :offset: how much to move each shift by (determines how many copies will be make)
    """
    curr_date = None
    stream = None

    for dat, row in gen_row_date(df):
        if dat != curr_date:
            try:
                stream_path = date_paths[dat]
            except KeyError:
                print(f"{dat} not in the stream_path")
                continue
                
            stream = stream_path.load()
            curr_date = dat
            
            
        # If shifting...
        for i in range(-1 * shift_left, shift_right+1, offset):
            yield filt.filter_waveform(stream, UTCDateTime(row.origintime) + i, pre_padding, post_padding)

            
        else:
            # Start PRE_PADDING before event_time, end POST_PADDING after event_time
            yield filt.filter_waveform(stream, UTCDateTime(row.origintime), pre_padding, post_padding)
        
def gen_filter_waves_from_times(times, date_paths, pre_padding=PRE_PADDING, post_padding=POST_PADDING):
    curr_date = None
    stream = None
    i = 0
    for time in times:
        dat = date(int(time.year), int(time.month), day=1)
        if dat != curr_date:
            try:
                stream_path = date_paths[dat]
            except KeyError:
                print(f"{dat} not in the stream_path ({i})", end='\r')
                i += 1
                continue
                
            stream = stream_path.load()
            curr_date = dat
            
        # Start PRE_PADDING before event_time, end POST_PADDING after event_time
        yield filt.filter_waveform(stream, time, pre_padding, post_padding)


In [17]:
date_paths = s1_date_paths
amount_quakes_ = None   # Write all events if None
amount_noise = 100000

In [18]:
import warnings
quake_path = os.path.join(SPECTROGRAM_PATH, 'local')
noise_path = os.path.join(SPECTROGRAM_PATH, 'noise')

### Write Quakes

In [56]:
# 8 additional copies will be made
shift_left = 5
shift_right = 4
total_shift = 9
assert shift_left + shift_right == total_shift

# To correct for the duplicate quakes
if amount_quakes_ is None:
    amount_quakes = None
else:
    amount_quakes = amount_quakes_ * (total_shift + 1) 
    
# Create Quakes
quake_waves = gen_filter_waves(quake_csv, date_paths, shift_left=shift_left, shift_right=shift_right)

In [ ]:
# Write Quakes
with warnings.catch_warnings():   
    warnings.simplefilter("ignore")
    spectro.async_write_spectrograms(islice(quake_waves, amount_quakes), 
                                     quake_path, ignoreexceptions=True, 
                                     write_streams=True,
                                     continue_from_previous=True)

Writing Files...


### Write Noise

In [19]:
def get_csv_times(df):
    times = df[['origintime']]
    for row in times.iterrows():
        time = row[1].origintime
        yield UTCDateTime(time)

times = sorted(get_csv_times(quake_csv))

In [20]:
noise_times = helpers.get_noise_times(times_to_exclude=times, 
                                      startafter=times[0], 
                                      endbefore=times[-1],
                                      amount=amount_noise, 
                                      duration=DURATION,
                                     )

In [21]:
noise_times = sorted(noise_times)  # important, to make lazy_loading the streams better for the next function

In [22]:
# Noise times centers around the given time by default... so pad X seconds in each direction
noise_pad = DURATION / 2
noise_waves = gen_filter_waves_from_times(noise_times, date_paths, noise_pad, noise_pad)

In [ ]:
with warnings.catch_warnings():   
    warnings.simplefilter("ignore")
    spectro.async_write_spectrograms(noise_waves, noise_path, ignoreexceptions=False, 
                                     write_streams=True,
                                     continue_from_previous=True)

Writing Files...


### Clean Up

In [ ]:
def clean_up(path):
    """ Removes empty dirs """
    folders = glob.glob(os.path.join(path, '*/'))
    for folder in folders:
        if not os.listdir(folder):
             os.rmdir(folder)

clean_up(quake_path)
clean_up(noise_path)